In [1]:
import jetson.inference
import jetson.utils
import ipywidgets 
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg as col
import cv2
import math
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt

In [2]:
def is_close(p1, p2):
    dst = math.sqrt(p1 ** 2 + p2 ** 2)
    return dst 
def convertBack(x, y, w, h): 
    #================================================================
    # 2.Purpose : Converts center coordinates to rectangle coordinates
    #================================================================  
    """
    :param:
    x, y = midpoint of bbox
    w, h = width, height of the bbox
    
    :return:
    xmin, ymin, xmax, ymax
    """
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax


In [30]:
def cvDrawBoxes(detections, img,distthresh):
    """
    :param:
    detections = total detections in one frame
    img = image from detect_image method of darknet
    :return:
    img with bbox
    """
    #================================================================
    # 3.1 Purpose : Filter out Persons class from detections and get 
    #           bounding box centroid for each person detection.
    #================================================================
    if len(detections) > 0:  						# At least 1 detection in the image and check detection presence in a frame  
        centroid_dict = dict() 						# Function creates a dictionary and calls it centroid_dict
        objectId = 0								# We inialize a variable called ObjectId and set it to 0
        for detection in detections:				# In this if statement, we filter all the detections for persons only
            # Check for the only person name tag 
            name_tag = detection.ClassID
            if name_tag != 1:                
                x, y, w, h = detection.Center[0],\
                            detection.Center[1],\
                            detection.Width,\
                            detection.Height      	# Store the center points of the detections
                xmin, ymin, xmax, ymax = convertBack(float(x), float(y), float(w), float(h))   # Convert from center coordinates to rectangular coordinates, We use floats to ensure the precision of the BBox            
                # Append center point of bbox for persons detected.
                centroid_dict[objectId] = (int(x), int(y), xmin, ymin, xmax, ymax) # Create dictionary of tuple with 'objectId' as the index center points and bbox
                objectId += 1 #Increment the index for each detection      
    #=================================================================#
    
    #=================================================================
    # 3.2 Purpose : Determine which person bbox are close to each other
    #=================================================================            	
        red_zone_list = [] # List containing which Object id is in under threshold distance condition. 
        red_line_list = []
        for (id1, p1), (id2, p2) in combinations(centroid_dict.items(), 2): # Get all the combinations of close detections, #List of multiple items - id1 1, points 2, 1,3
            dx, dy = p1[0] - p2[0], p1[1] - p2[1]  	# Check the difference between centroid x: 0, y :1
            distance = is_close(dx, dy) 			# Calculates the Euclidean distance
            if distance <= distthresh:						# Set our social distance threshold - If they meet this condition then..
                if id1 not in red_zone_list:
                    red_zone_list.append(id1)       #  Add Id to a list
                    red_line_list.append(p1[0:2])   #  Add points to the list
                if id2 not in red_zone_list:
                    red_zone_list.append(id2)		# Same for the second id 
                    red_line_list.append(p2[0:2])
        
        for idx, box in centroid_dict.items():  # dict (1(key):red(value), 2 blue)  idx - key  box - value
            if idx in red_zone_list:   # if id is in red zone list
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (255, 0, 0), 2) # Create Red bounding boxes  #starting point, ending point size of 2
            else:
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (0, 255, 0), 2) # Create Green bounding boxes
		#=================================================================#

		#=================================================================
    	# 3.3 Purpose : Display Risk Analytics and Show Risk Indicators
    	#=================================================================        
        text = "People at Risk: %s" % str(len(red_zone_list)) 			# Count People at Risk
        
        location = (10,25)												# Set the location of the displayed text
        cv2.putText(img, text, location, cv2.FONT_HERSHEY_SIMPLEX, 1, (246,86,86), 2, cv2.LINE_AA)  # Display Text

        for check in range(0, len(red_line_list)-1):					# Draw line between nearby bboxes iterate through redlist items
            start_point = red_line_list[check] 
            end_point = red_line_list[check+1]
            check_line_x = abs(end_point[0] - start_point[0])   		# Calculate the line coordinates for x  
            check_line_y = abs(end_point[1] - start_point[1])			# Calculate the line coordinates for y
            #if (check_line_x < distthresh) and (check_line_y < 25):				# If both are We check that the lines are below our threshold distance.
            cv2.line(img, start_point, end_point, (255, 0, 0), 2)   # Only above the threshold lines are displayed. 
        #=================================================================#
    return img

In [4]:
import depthai as dai

# Create pipeline
pipeline = dai.Pipeline()

# Define source and output
camRgb = pipeline.createColorCamera()
xoutRgb = pipeline.createXLinkOut()

xoutRgb.setStreamName("rgb")

# Properties
camRgb.setPreviewSize(1280, 720)
camRgb.setInterleaved(False)
camRgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Linking
camRgb.preview.link(xoutRgb.input)

# Connect to device and start pipeline
#with dai.Device(pipeline) as device:

device = dai.Device(pipeline)

print('Connected cameras: ', device.getConnectedCameras())
# Print out usb speed
print('Usb speed: ', device.getUsbSpeed().name)

# Output queue will be used to get the rgb frames from the output defined above
qRgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)


inRgb = qRgb.get()  # blocking call, will wait until a new data has arrived

 # Retrieve 'bgr' (opencv format) frame
#cv2.imshow("rgb", inRgb.getCvFrame())
pic = inRgb.getCvFrame()
img = jetson.utils.cudaFromNumpy(pic)
image_widget = ipywidgets.Image(format = 'jpeg')
        

array = jetson.utils.cudaToNumpy(img)
    
image_widget.value = col(array)
display(image_widget)
   

Connected cameras:  [<CameraBoardSocket.RGB: 0>, <CameraBoardSocket.LEFT: 1>, <CameraBoardSocket.RIGHT: 2>]
Usb speed:  SUPER


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [27]:
net = jetson.inference.detectNet("ped-100",threshold=0.5)

camera = jetson.utils.videoSource("/dev/video0")      # '/dev/video0' for V4L2
dis = jetson.utils.videoOutput("display://0") # 'my_video.mp4' for file

In [28]:
image_widget = ipywidgets.Image(format = 'jpeg')
img = camera.Capture()

array = jetson.utils.cudaToNumpy(img)

image_widget.value = col(array)
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [32]:
while dis.IsStreaming():
    # blocking call, will wait until a new data has arrived

    # Retrieve 'bgr' (opencv format) frame
    #cv2.imshow("rgb", inRgb.getCvFrame())
    img = camera.Capture()
    
    detections = net.Detect(img,overlay = 'none')

    array = jetson.utils.cudaToNumpy(img)
    cvDrawBoxes(detections, array,1000)
    image_widget.value = col(array)
    #display.SetStatus("Object Detection | Network {:.0f} FPS".format(net.GetNetworkFPS()))

KeyboardInterrupt: 

In [6]:
while dis.IsStreaming():
    inRgb = qRgb.get()  # blocking call, will wait until a new data has arrived

    # Retrieve 'bgr' (opencv format) frame
    #cv2.imshow("rgb", inRgb.getCvFrame())
    pic = inRgb.getCvFrame()
    img = jetson.utils.cudaFromNumpy(pic)
    array = jetson.utils.cudaToNumpy(img)
    image_widget.value = col(array)

KeyboardInterrupt: 